In [1]:
import os

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datetime import datetime
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [3]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GENERATIVE_LANGUAGE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [5]:
def transfer_to_calendar_agent(input_str):
    return calendar_agent_executor.invoke({"input": input_str})
def transfer_to_travel_agent(input_str):
    return travel_agent_executor.invoke({"input": input_str})

In [6]:
from planing_tools import weatherapi_forecast_periods, query_rag
ddg_search = DuckDuckGoSearchAPIWrapper()

travel_planing_tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="""Essa ferramenta DEVE ser utilizada para buscar eventos relevantes no período fornecido pelo usuário. 
        Ela é útil para obter informações sobre eventos ou atividades especiais que estão acontecendo na cidade de destino nas datas que o usuário informou. 
        O modelo deve usá-la para complementar as sugestões de atividades."""
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="""Essa ferramenta DEVE ser usada *antes* de gerar o roteiro turístico, após o modelo coletar todas as informações necessárias do usuário, incluindo as datas exatas. 
    O modelo deve consultar o clima separadamente para cada dia do período informado, garantindo que as atividades do roteiro sejam planejadas de acordo com as condições climáticas. 
    A previsão do tempo é um passo obrigatório quando é um fator relevante para o planejamento."""
    ),
    Tool(
        name="Query RAG",
        func=query_rag,
        description="""Esta ferramenta deve ser usada quando o modelo souber a cidade de destino e os interesses do usuário, com o objetivo de fornecer informações sobre pontos turísticos e atrações que se alinham com esses interesses. 
        O modelo deve utilizar essa ferramenta para sugerir atividades e lugares específicos a visitar, baseados na cidade e nos interesses fornecidos."""
    ),
    Tool(
        name="Calendar Agent",
        func=transfer_to_calendar_agent,
        description="Esse agente lida com tudo relacionado ao calendário."
    ),
]

In [7]:
from calendar_tools import list_calendar_list, list_calendar_events, insert_calendar_event, create_calendar

google_calendar_tools = [
    Tool(
        name="Create Calendar List",
        func=create_calendar,
        description="""
        Use a função create_calendar_list para criar um novo calendário.
        Forneça o resumo (título) do calendário que você deseja criar.
        - Exemplo de uso: 'Meu Calendário de Viagens'
        - Essa função vai criar um novo calendário com o resumo fornecido.
        """
    ),
    Tool(
        name="List Calendar List",
        func=list_calendar_list,
        description="""
        Use a função list_calendar_list para obter a lista dos seus calendários disponíveis no Google Calendar.
        Se você quiser limitar o número de calendários retornados, forneça o número máximo desejado.
        - Exemplo de uso: Forneça o número máximo de calendários a serem listados, como 50. Se não quiser um limite, apenas diga para listar 200 o máximo por padrão.
        """
    ),
    Tool(
        name="List Calendar Events",
        func=list_calendar_events,
        description="""
        Use a função list_calendar_events para obter a lista de eventos de um calendário específico.
        Você precisa fornecer o ID do calendário e, opcionalmente, o número máximo de eventos a serem listados.
        - Exemplo de uso para o calendário principal: forneça 'primary' como o ID do calendário.
        - Exemplo de uso com limite: forneça o ID do calendário e o número máximo de eventos, como 'primary', 20.
        - Para listar eventos de um calendário específico, primeiro liste seus calendários para encontrar o ID correto.
        """
    ),
    Tool(
        name="Insert Calendar Event",
        func=insert_calendar_event,
        description="""
        Use a função Insert Calendar Event para adicionar um evento a um calendário.
        Você precisa fornecer um dicionário Python contendo os detalhes do evento.
        Se o timezone não for mencionado utilize "America/Fortaleza"
        Se o usuário informar o nome do calendário para inserir o evento chame a função list_calendar_list primeiro para adquirir a informação de calendar_id
        O dicionário deve ter a seguinte estrutura:
        {
            "calendar_id": "primary",
            "summary": "Jantar em restaurante regional em Ponta Negra",
            "location": "Restaurante regional em Ponta Negra",
            "description": "",
            "start": {
                "dateTime": "2024-01-01T19:00:00",
                "timeZone": "America/Fortaleza"
            },
            "end": {
                "dateTime": "2024-01-01T22:00:00",
                "timeZone": "America/Fortaleza"
            },
            "attendees": []
        }
        Ao final você DEVE fornecer o link do evento criado ao usuário além das informações do evento criado.
        """
    ),
    Tool(
        name="Main Agent",
        func=transfer_to_travel_agent,
        description="Transfere ao agente de turismo para perguntas não relacionadas a calendário"
    )
]

calendar v3 service created successfully


In [8]:
travel_planing_prompt = hub.pull("tales/agente_turismo")
google_calendar_prompt = hub.pull("tales/agente_calendario")
# main_agent_prompt = hub.pull("tales/agente_principal")

In [9]:
planing_prompt = travel_planing_prompt.partial(
    tools=render_text_description(travel_planing_tools),
    tool_names=", ".join([t.name for t in travel_planing_tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [10]:
calendar_prompt = google_calendar_prompt.partial(
    tools=render_text_description(google_calendar_tools),
    tool_names=", ".join([t.name for t in google_calendar_tools]),
)

In [11]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_1308\275711510.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")


In [12]:
travel_planing_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | planing_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

travel_agent_executor = AgentExecutor(agent=travel_planing_agent, tools=travel_planing_tools, verbose=True, memory=memory)

In [13]:
google_calendar_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | calendar_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

calendar_agent_executor = AgentExecutor(agent=google_calendar_agent, tools=google_calendar_tools, verbose=True, memory=memory)

In [14]:
response = travel_agent_executor.invoke({"input": 'Boa noite'})

print(response["output"])

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Boa noite! Para começarmos a planejar sua viagem, qual cidade ou região você tem em mente?


> Finished chain.
Boa noite! Para começarmos a planejar sua viagem, qual cidade ou região você tem em mente?


In [15]:
response = travel_agent_executor.invoke({"input": 'Eu estou planejando visitar Natal no Rio Grande do Norte'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma excelente escolha.  Quantos dias você pretende passar lá?


> Finished chain.
Ótimo! Natal é uma excelente escolha.  Quantos dias você pretende passar lá?


In [16]:
response = travel_agent_executor.invoke({"input": 'Eu planejo passar 2 dias da quarta dia 01 de janeiro e quinta'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Perfeito! Dois dias em Natal entre 01 e 02 de Janeiro.  Para me ajudar a criar o roteiro ideal, quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


> Finished chain.
Perfeito! Dois dias em Natal entre 01 e 02 de Janeiro.  Para me ajudar a criar o roteiro ideal, quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


In [17]:
response = travel_agent_executor.invoke({"input": 'Eu planejo visitar praias e restaurantes regionais'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Entendido! Praias e restaurantes regionais.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?


> Finished chain.
Entendido! Praias e restaurantes regionais.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?


In [18]:
response = travel_agent_executor.invoke({"input": 'Em torno de 400 reais'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo! Um orçamento de R$400 para dois dias.  Você segue alguma dieta específica ou tem restrições alimentairas, como alergias, que devo levar em consideração?


> Finished chain.
Ótimo! Um orçamento de R$400 para dois dias.  Você segue alguma dieta específica ou tem restrições alimentairas, como alergias, que devo levar em consideração?


In [19]:
response = travel_agent_executor.invoke({"input": 'Não possuo nenhuma restrição alimentar.'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 01/01/2025, 02/01/2025Previsão para 01/01/2025, 02/01/2025 em Natal:

Manhã:
Hora: 05:00 - Temp: 26.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 79%
Hora: 06:00 - Temp: 27.5°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 74%
Hora: 07:00 - Temp: 28.2°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 70%
Hora: 08:00 - Temp: 29.1°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 66%
Hora: 09:00 - Temp: 29.8°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 79%, Umidade: 64%
Hora: 10:00 - Temp: 30.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 81%, Umidade: 61%
Hora: 11:00 - Temp: 30.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 70%, Umidade: 61%

Tarde:
Hora: 13:00 - Temp: 30.1°C, Condição: Possibilidade de chuva irregular, 

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: melhores praias e restaurantes em Natal, Rio Grande do NortePRAIA A PRAIA
A orla urbana de Natal é marcada por completa estrutura 
hoteleira, com quiosques atendendo à beira-mar. Da movimentada 
Ponta Negra à tradicional Redinha, tem praia para todos os gostos!
VIA COSTEIRA
A continuação da Praia de Ponta Negra acompanha a famosa Via Costeira, principal 
avenida da cidade. Nessa faixa de areia, ficam os incríveis resorts da cidade, com 
estrutura completa de lazer e gastronomia. A bela paisagem da região é pontuada por 
dunas, trechos de Mata Atlântica e também lindas praias desertas.
PONTA NEGRA
O Morro do Careca é o marco principal da Praia de Ponta Negra e cartão-postal de 
Natal. Emoldurada por vegetação, a duna de 107 m de altura domina a paisagem – 
mas é proibido subir nela por questões ambientais. A praia em si é supermovimentada,

---

PRAIA DA REDINHA
A última praia ao norte de Natal é uma anti

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Olá! Com base nas informações que você me deu, preparei um roteiro para sua viagem de dois dias a Natal, entre 01 e 02 de janeiro de 2025, focando em praias e restaurantes regionais, com um orçamento de R$400 e sem restrições alimentares.  Considerando a previsão do tempo, que indica chuva em alguns períodos,  sugiro algumas alternativas para garantir que aproveite ao máximo sua viagem.


**Roteiro de Viagem: Natal (01 e 02 de Janeiro de 2025)**

**Dia 01 de Janeiro (Quarta-feira):**

* **Manhã (08:00 - 12:00):**  Como a previsão indica chuva pela manhã, sugiro começar o dia visitando o Aquário de Natal, na Praia da Redinha.  É uma ótima opção para se proteger da chuva e aprender sobre a vida marinha da região.  O custo aproximado é de R$ 40,00.

* **Tarde (13:00 - 17:00):** Se a chuva parar, aproveite para visitar a Praia de Ponta Negra.  Caso a chuva persista, visite o Parque das Dunas.  O Parque oferece trilhas e um mir

In [20]:
response = travel_agent_executor.invoke({"input": 'Eu quero que você salve os eventos sugeridos no meu calendário google'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Calendar Agent
Action Input: Criar eventos no Google Calendar para o roteiro de viagem a Natal entre 01 e 02 de Janeiro de 2025:

**Dia 01 de Janeiro (Quarta-feira):**

* **Manhã (08:00 - 12:00):** Aquário de Natal (Praia da Redinha) - Alternativa em caso de chuva.
* **Tarde (13:00 - 17:00):** Praia de Ponta Negra (se tempo bom) ou Parque das Dunas (se chuva).
* **Noite (18:00 - 22:00):** Jantar em restaurante regional na Praia de Ponta Negra e passeio pela orla (se tempo bom).

**Dia 02 de Janeiro (Quinta-feira):**

* **Manhã (08:00 - 12:00):** Praia do Forte ou Praia dos Artistas (dependendo do tempo) ou Museu de Arte Sacra (se chuva).
* **Tarde (13:00 - 17:00):** Explorar a Via Costeira e almoçar em restaurante ou quiosque.
* **Noite (18:00 - 22:00):** Jantar em restaurante regional na Via Costeira.

Observações:  Os locais específicos de restaurantes precisam ser definidos pelo usuário.  O transporte entre os locais deve se

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Create Calendar List
Action Input: Natal Trip 2025{'kind': 'calendar#calendar', 'etag': '"CObZDZeyFpiTs4hRp1ruEq8lMDg"', 'id': '2ed5c11fca0dbc8d66c04f273763b3ae6fb88f119ca6e2e7f28b00b37c9155e5@group.calendar.google.com', 'summary': 'Natal Trip 2025', 'timeZone': 'UTC', 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

...

Thought: Do I need to use a tool? Yes
Action: Insert Calendar Event
Action Input: {
            "calendar_id": "2ed5c11fca0dbc8d66c04f273763b3ae6fb88f119ca6e2e7f28b00b37c9155e5@group.calendar.google.com",
            "summary": "Jantar em restaurante regional na Via Costeira",
            "location": "Restaurante regional na Via Costeira, Natal - RN",
            "description": "Jantar na Via Costeira.",
            "start": {
                "dateTime": "2025-01-02T18:00:00",
                "timeZone": "America/Fortaleza"
            },
            "end": {
                "dateTime": "2025-01-02T22:00:00",
                "timeZone": "America/Fortaleza"
            },
            "attendees": []
        }{'kind': 'calendar#event', 'etag': '"3471041171546000"', 'id': 'ppjkq1quhur6cahg1ou4m60lp4', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=cHBqa3ExcXVodXI2Y2FoZzFvdTRtNjBscDQgMmVkNWMxMWZjYTBkYmM4ZDY2YzA0ZjI3Mzc2M2IzYWU2ZmI4OGYxMTljYTZlMmU3ZjI4YjAwYjM3

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo!  Criei um calendário chamado "Natal Trip 2025" e adicionei os eventos da sua viagem.  Segue a lista de eventos e seus links:

**Dia 01 de Janeiro (Quarta-feira):**

* **Manhã (08:00 - 12:00):** Aquário de Natal: [https://www.google.com/calendar/event?eid=Mjc3ZDB0dWsxOThlZmE3MjU4YnRsZXFiZW8gMmVkNWMxMWZjYTBkYmM4ZDY2YzA0ZjI3Mzc2M2IzYWU2ZmI4OGYxMTljYTZlMmU3ZjI4YjAwYjM3YzkxNTVlNUBn](https://www.google.com/calendar/event?eid=Mjc3ZDB0dWsxOThlZmE3MjU4YnRsZXFiZW8gMmVkNWMxMWZjYTBkYmM4ZDY2YzA0ZjI3Mzc2M2IzYWU2ZmI4OGYxMTljYTZlMmU3ZjI4YjAwYjM3YzkxNTVlNUBn)
* **Tarde (13:00 - 17:00):** Praia de Ponta Negra ou Parque das Dunas: [https://www.google.com/calendar/event?eid=MTkxdDBmOTlxbGEzNDZoczYxOGUyN2NxYWsgMmVkNWMxMWZjYTBkYmM4ZDY2YzA0ZjI3Mzc2M2IzYWU2ZmI4OGYxMTljYTZlMmU3ZjI4YjAwYjM3YzkxNTVlNUBn](https://www.google.com/calendar/event?eid=MTkxdDBmOTlxbGEzNDZoczYxOGUyN2NxYWsgMmVkNWMxMWZjYTBkYmM4ZDY2YzA0ZjI3Mzc2M2IzYWU2ZmI4OGYxMTljYTZlMmU3ZjI4YjAwYjM

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo!  Criei um calendário chamado "Natal Trip 2025" e adicionei os eventos da sua viagem.  Você receberá uma notificação com os detalhes e links para cada evento.  Lembre-se que este é um roteiro sugerido e você pode ajustá-lo conforme necessário.  Aproveite sua viagem para Natal!
```


> Finished chain.
Ótimo!  Criei um calendário chamado "Natal Trip 2025" e adicionei os eventos da sua viagem.  Você receberá uma notificação com os detalhes e links para cada evento.  Lembre-se que este é um roteiro sugerido e você pode ajustá-lo conforme necessário.  Aproveite sua viagem para Natal!
```


In [21]:
print(response["chat_history"])

Human: Boa noite
AI: Boa noite! Para começarmos a planejar sua viagem, qual cidade ou região você tem em mente?
Human: Eu estou planejando visitar Natal no Rio Grande do Norte
AI: Ótimo! Natal é uma excelente escolha.  Quantos dias você pretende passar lá?
Human: Eu planejo passar 2 dias da quarta dia 01 de janeiro e quinta
AI: Perfeito! Dois dias em Natal entre 01 e 02 de Janeiro.  Para me ajudar a criar o roteiro ideal, quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```
Human: Eu planejo visitar praias e restaurantes regionais
AI: Entendido! Praias e restaurantes regionais.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?
Human: Em torno de 400 reais
AI: Ótimo! Um orçamento de R$400 para dois dias.  Você segue alguma dieta específica ou tem restrições alimentairas, como alergias, que devo levar em co